# Лабораторная работа 3: Машина опорных векторов (SVM)

<img src="svm_img.png" width=350>

Результат лабораторной работы — **отчет** в формате ноутбуков IPython (ipynb-файл). Нам не интересен ваш код. Чем меньше кода, тем лучше всем: нам — меньше проверять, вам — проще найти ошибку или дополнить эксперимент.

Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Ответы на вопросы должны быть полными, четкими и хорошо аргументированными.

## 1. Квадратичное программирование и QP-солвер

Квадратичное программирование (QP) — специальный тип задач математической оптимизации, заключающийся в нахождении точки минимума неотрицательно-определенной квадратичной формы (многомерная парабола) в присутствии линейных ограничений:
$$
\begin{gather}
    \frac{1}{2}\boldsymbol x^T\boldsymbol P\boldsymbol x + \boldsymbol q^T\boldsymbol x \to \min_{\boldsymbol x} \\
    \begin{aligned}
        \text{s.t.} \quad & \boldsymbol G\boldsymbol x \le \boldsymbol h \\
        & \boldsymbol A\boldsymbol x = \boldsymbol b
    \end{aligned}
\end{gather}
$$

Здесь $\boldsymbol P$ — симметричная матрица. В ограничениях $\boldsymbol G\boldsymbol x \le \boldsymbol h$ и $\boldsymbol A\boldsymbol x = \boldsymbol b$ под знаками равенства и неравенства подразумевается сравнение всех компонент векторов.

Задача квадратичного программирования хорошо изучена, существуют эффективные алгоритмы для ее решения. Имеется множество библиотек с солверами для задачи QP, вот некоторые из них:
 - [CVXOPT](http://cvxopt.org/) (свободно-распространяемая, Python) **рекомендуется к использованию**
 - [Mosek](https://www.mosek.com/) (коммерческая с возможностью получения академической лицензии, C, Java, MATLAB, .NET, R, Python)
 - [Matlab Optimization Toolbox](http://www.mathworks.com/help/optim/ug/quadprog.html) 

#### Задание

1. Установите QP-солвер, разберитесь с его интерфейсом.
2. Протестируйте солвер, решив простую задачу оптимизации в двумерном пространстве:
$$f(\boldsymbol x) = -8x_1 - 16x_2 + x_1^2 + 4x_2^2, \quad \text{s.t.:} \; x_1 + x_2 \leq 5, \; 0 \leq x_1 \leq 3, \; x_2 \geq 0$$
3. Какие из ограничений-неравенств задачи являются _активными_, т.е. влияют на точку оптимума, а какие ограничения можно выбросить и точка оптимума не поменяется?

In [2]:
pip install cvxopt

  Obtaining dependency information for cvxopt from https://files.pythonhosted.org/packages/a3/52/2237d72cf007e6c36367ab8a776388a9f13511e4cfa8a71b79101ad6e0fa/cvxopt-1.3.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 12.8/12.8 MB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import PolynomialFeatures
from cvxopt import matrix, solvers

P = matrix(np.array([[2.0, 0.0],
                     [0.0, 8.0]]))

q = matrix(np.array([-8.0, -16.0]))

G = matrix(np.array([[-1.0, 0.0], 
                     [1.0, 0.0], 
                     [0.0, -1.0], 
                     [1.0, 1.0]]))

h = matrix(np.array([0.0, 3.0, 0.0, 5.0]))

sol = solvers.qp(P, q, G, h)

     pcost       dcost       gap    pres   dres
 0: -3.0512e+01 -3.8829e+01  8e+00  0e+00  2e-01
 1: -3.0792e+01 -3.1146e+01  4e-01  2e-16  5e-03
 2: -3.0982e+01 -3.1024e+01  4e-02  1e-16  4e-04
 3: -3.0997e+01 -3.1003e+01  6e-03  2e-16  5e-17
 4: -3.1000e+01 -3.1000e+01  7e-04  2e-16  3e-17
 5: -3.1000e+01 -3.1000e+01  1e-04  2e-16  4e-17
 6: -3.1000e+01 -3.1000e+01  1e-05  2e-16  3e-17
Optimal solution found.


In [15]:
print(sol["x"])

[ 3.00e+00]
[ 2.00e+00]



In [16]:
G = matrix(np.array([[1.0, 0.0]]))

h = matrix(np.array([3.0]))

sol = solvers.qp(P, q, G, h)

     pcost       dcost       gap    pres   dres
 0: -3.1889e+01 -3.1444e+01  7e-01  6e-01  1e-16
 1: -3.1639e+01 -3.1160e+01  1e-02  1e-01  4e-17
 2: -3.0992e+01 -3.1000e+01  8e-03  0e+00  1e-16
 3: -3.1000e+01 -3.1000e+01  8e-05  1e-16  0e+00
 4: -3.1000e+01 -3.1000e+01  8e-07  0e+00  2e-17
Optimal solution found.


In [17]:
print(sol["x"])

[ 3.00e+00]
[ 2.00e+00]



Ограничение: $x \le 3$

## 2. Линейный SVM

Рассмотрим задачу бинарной классификации. Будем обозначать обучающую выборку $\{(\boldsymbol x_n, y_n)\}_{n=1}^N$, где $N$ — количество объектов, $\boldsymbol x_n \in \mathbb{R}^d$ — числовой вектор признакового описания объекта, $y_n \in \{+1, -1\}$ — класс объекта.

Машина опорных векторов обучает модель разделяющей гиперплоскости:
$$f(\boldsymbol x) = \boldsymbol w^T \boldsymbol x + b$$
Параметры модели — вектор весов $\boldsymbol w \in \mathbb{R}^d$ и сдвиг $b \in \mathbb{R}$.

Обучение модели происходит путем решения оптимизационной задачи:
$$
\begin{gather}
    \frac{1}{2} \| \boldsymbol w \|^2 + C \sum_{n=1}^N \xi_n \to \min_{\boldsymbol w, \boldsymbol \xi, b} \\
    \text{s.t.: } \quad y_n (\boldsymbol w^T \boldsymbol x_n + b) \geq 1 - \xi_n, \quad \xi_n \geq 0, \quad \forall n=1,\dots,N
\end{gather}
$$

Ограничения вида $\quad y_n (\boldsymbol w^T \boldsymbol x_n + b) \geq 1$ требуют, чтобы объекты правильно классифицировались разделяющей гиперплоскостью. Поскольку линейная разделимость выборки не гарантируется на практике, вводят переменные $\xi_n$ (slack variables), которые ослабляют ограничения правильной классификации. В оптимизируемом функционале слагаемое $\| \boldsymbol w \|^2$ штрафует малую ширину разделяющей гиперплоскости, сумма $\sum_n \xi_n$ штрафует ослабление ограничений. 

Гиперпараметр $C$ задает баланс между шириной разделяющей полосы и ошибками, допускаемыми классификатором. Обратите внимание, что $C$ фиксируется до обучения и не оптимизируется вместе с параметрами модели.

Гиперпараметр $C$ отвечает за обобщающую способность разделяющей гиперплоскости, высокая обобщающая способность (соотвествующая большому значению $C$) может привести к переобучению, если линейная модель хорошо описывает обучающие примеры. При подборе оптимального параметра $C$ необходимо оценивать качество на отложенной выборке или кросс-валидации. Как правило, для конкретной задачи заранее неизвестно, какой порядок имеет оптимальное значение гиперпараметра $C$, поэтому перебирать значения лучше по логарифмической сетке, например: $10^{-3}, 10^{-2}, \dots, 10^{5}$.

После нахождения решения оптимизационной задачи $(\boldsymbol w_{\star}, \boldsymbol \xi_{\star}, b_{\star})$, часть ограничений становятся _активными_, т.е. переходят в "крайнее положение" — точное равенство:
$$\quad y_n (\boldsymbol w_{\star}^T \boldsymbol x_n + b_{\star}) = 1 - \xi_{\star,n}$$
Объекты, соответствующие активным ограничениям называются _опорными_.

#### Явное преобразование признаков

Разделяющая гиперплоскость — достаточно грубая модель, редко данные могут быть описаны линейной моделью. Линейная неразделимость векторов может быть исправлена путем перехода в другое признаковое пространство, в котором линейная модель лучше описывает данные и, возможно, существует правильно классифицирующая разделяющая гиперплоскость:

$$\boldsymbol x \in \mathbb{R}^d \mapsto \phi(\boldsymbol x) \in \mathbb{R}^t$$

Так, например, добавление всех попарных произведений признаков: $\phi(x_1, \dots, x_d) = (x_1, \dots, x_d, x_1^2, x_1x_2, \dots, x_d^2)$ переводит в пространство, в котором линейная гиперплоскость является квадратичной формой в исходном пространстве и в исходном пространстве признаков разделяющая поверхность может быть, скажем, эллипсом.

[Видеоролик с демонстрацией](https://youtu.be/9NrALgHFwTo)

#### Задание
  1. Сведите задачу обучения линейного SVM к QP, реализуйте процедуру обучения Линейного SVM при помощи QP-солвера.
  2. Сгенерируйте три случайные двумерные выборки для бинарной классификации:
    - с линейно-разделимыми классами
    - с хорошо разделимыми классами, но не линейно
    - с плохо разделимыми классами по имеющимся признакам
  3. Протестируйте линейный SVM на сгенерированных выборках. Покажите на плоскости разделяющую прямую и линии уровня, ограничивающие коридор $f(\boldsymbol x) = \pm 1$. Выделите опорные вектора точками другой формы или большего размера. Постройте классификаторы с различным значением параметра $C$.
  4. Как зависит число опорных векторов от параметра $C$ для различных выборок?
  5. Используя явное преобразование признаков обучите методом опорных векторов квадратичную разделяющую поверхность. Покажите ее на плоскости.

In [18]:
def hyperplane(w, x, b):
    result = 0
    for i in range(len(w)-1):
        result -= w[i] * x[i]
    result /= w[-1] + np.pow(10, -10)
    return result

def square_transform(X):
    '''
    f(x_1, ..., x_d) = (x_1, ... x_d, x_1^2, x_1x_2, ..., x_d^2)
    '''
    poly = PolynomialFeatures(2)
    X_new = poly.fit_transform(X)
    X_new = X_new[:,1:]
    #X_new = np.column_stack((X, X[:, 0] ** 2 + X[:, 1] ** 2))
    
    return X_new

def cube_transform(X):
    '''
    f(x_1, ..., x_d) = (x_1, ... x_d, x_1^2, x_1x_2, ..., x_d^2)
    '''
    poly = PolynomialFeatures(3)
    X_new = poly.fit_transform(X)
    X_new = X_new[:,1:]
    #X_new = np.column_stack((X, X[:, 0] ** 2 + X[:, 1] ** 2))
    
    return X_new

In [19]:
a = np.linspace(0, 9, 10)
b = np.linspace(0, 9, 10)

xx, yy = np.meshgrid(a, b)
#print(xx, yy)
#plt.scatter(xx, yy)
s = np.column_stack((xx.ravel(), yy.ravel()))
s = square_transform(s)
s[0]

array([0., 0., 0., 0., 0.])

In [20]:
def dot_line_distanse(coeff, free_coeff, x):

    numerator, denominator  = 0, 0
    for i in range(len(coeff)):
        numerator += coeff[i] * x[i]
    numerator += free_coeff

    for i in range(len(coeff)):
        denominator += coeff[i] ** 2
    
    return abs(numerator) / np.sqrt(denominator)

In [21]:
from sklearn.base import BaseEstimator

class LinearSVM(BaseEstimator):
    def __init__(self, C, transform=None):
        self.C = C
        self.transform = transform
        
    def fit(self, X, y):
        """
        Функция обучения модели.
        """
        M, n = X.shape
        self.weights_ = np.empty(M)
        self.bias_ = 0
        y[np.where(y==0)] = -1
        self.cords_ = X  # для .draw()
        self.labels_ = y
        
        self.zeros_ = np.zeros((M, M))
        P = np.diag(np.ones(n + 1))
        P[-1, -1] = 0
        P = np.hstack((P, np.zeros((n + 1, M))))
        P = np.vstack((P, np.zeros((M, M + n + 1))))

        q = np.concatenate((np.zeros(n + 1), np.ones(M) * self.C))
        
        G = np.concatenate(((-y * X.T).T, np.array([-y]).T), axis=1)
        G = np.vstack((G, np.zeros((M, n + 1))))
        self.doble_ones_ = np.vstack((np.diag(-np.ones(M)), np.diag(-np.ones(M))))
        G = np.hstack((G, self.doble_ones_))
        
        h = np.concatenate((-np.ones(M), np.zeros(M)))
        
        P = matrix(P, size=(n + 1 + M, n + 1 + M), tc='d')
        q = matrix(q, tc='d')
        G = matrix(G, size=(2 * M, n + 1 + M), tc='d')
        h = matrix(h, tc='d')


        solution = np.array(qp(P=P, q=q, G=G, h=h)['x'])
        self.weights_ = solution[:-M-1]
        self.bias_ = solution[n]
        #self.bias_ = np.median(np.dot(self.weights_, X) - y)
        self.M_, self.n_ = M, n
        #return self.weights_, self.bias_

        
    def predict_proba(self, X):
        return (np.dot(X, self.weights_) + self.bias_).flatten()
            
    def predict(self, X):
        return np.sign(self.predict_proba(X))

    def draw(self, ax_i, ax_j, *, title=None, axis_names=None):
        '''
        Plots cool beautiful shockingly magnificent graph.
        I spent 5 hours figuring this out, so yeah.

        Params:
        ax_i and ax_j is coordinates for picture in plt.subplots
        Title is title, and axis_names is the names of the axises
        
        '''

        xvals = np.linspace(self.cords_[:, 0].min() - 1, self.cords_[:, 0].max() + 1, int(self.M_))
        yvals = np.linspace(self.cords_[:, 1].min() - 1, self.cords_[:, 1].max() + 1, int(self.M_))

        space = np.meshgrid(xvals, yvals)
        #print(space)
        space = np.column_stack((space[0].ravel(), space[1].ravel()))
        if self.transform == 'square':
            space = square_transform(space)
        elif self.transform == 'cube':
            space = cube_transform(space)

        # zz = []
        # for m in range(self.M_ ** 2):
        #     zz.append(svm.predict_proba(space[m]))

        zz = self.predict_proba(space)
        xx, yy = xvals, yvals
        
        zz = np.array(zz).reshape(int(self.M_), int(self.M_))
        ax[ax_i, ax_j].pcolormesh(xx, yy, zz, shading='gouraud', cmap='viridis')

        margin = 1 / np.linalg.norm(self.weights_)

        ax[ax_i, ax_j].contour(xx, yy, zz, levels=(0), colors='k', linewidths=1.5, zorder=1)
        ax[ax_i, ax_j].contour(xx, yy, zz, levels=([-1, 1]), colors='white', linewidths=1.5, zorder=2, linestyles='dotted')
        
        ax[ax_i, ax_j].scatter(self.cords_[:, 0], self.cords_[:, 1], marker="o", c=self.labels_, s=25, cmap='viridis', edgecolor="k");
        
        support_vectors = []
        support_labels = []
        for i in range(self.M_):
            if dot_line_distanse(self.weights_, self.bias_, self.cords_[i]) <= margin + 10 ** -4:
                support_vectors.append(self.cords_[i])
                support_labels.append(self.labels_[i])
        support_vectors = np.array(support_vectors)

        if len(support_vectors):
            ax[ax_i, ax_j].scatter(support_vectors[:, 0], support_vectors[:, 1], marker="o", c=support_labels,
                                   s=25, cmap='viridis', edgecolor="1")
        
        ax[ax_i, ax_j].set_ylim(self.cords_[:, 1].min() - 1, self.cords_[:, 1].max() + 1)
        ax[ax_i, ax_j].set_title(title)

In [22]:
from sklearn.base import BaseEstimator

class LinearSVM(BaseEstimator):
    def __init__(self, C, transform=None):
        self.C = C
        self.transform = transform
        
    def fit(self, X, y):
        """
        Функция обучения модели.
        """
        M, n = X.shape
        self.weights_ = np.empty(M)
        self.bias_ = 0
        y[np.where(y==0)] = -1
        self.cords_ = X  # для .draw()
        self.labels_ = y
        
        self.zeros_ = np.zeros((M, M))
        P = np.diag(np.ones(n + 1))
        P[-1, -1] = 0
        P = np.hstack((P, np.zeros((n + 1, M))))
        P = np.vstack((P, np.zeros((M, M + n + 1))))

        q = np.concatenate((np.zeros(n + 1), np.ones(M) * self.C))
        
        G = np.concatenate(((-y * X.T).T, np.array([-y]).T), axis=1)
        G = np.vstack((G, np.zeros((M, n + 1))))
        self.doble_ones_ = np.vstack((np.diag(-np.ones(M)), np.diag(-np.ones(M))))
        G = np.hstack((G, self.doble_ones_))
        
        h = np.concatenate((-np.ones(M), np.zeros(M)))
        
        P = matrix(P, size=(n + 1 + M, n + 1 + M), tc='d')
        q = matrix(q, tc='d')
        G = matrix(G, size=(2 * M, n + 1 + M), tc='d')
        h = matrix(h, tc='d')


        solution = np.array(qp(P=P, q=q, G=G, h=h)['x'])
        self.weights_ = solution[:-M-1]
        self.bias_ = solution[n]
        #self.bias_ = np.median(np.dot(self.weights_, X) - y)
        self.M_, self.n_ = M, n
        #return self.weights_, self.bias_

        
    def predict_proba(self, X):
        return (np.dot(X, self.weights_) + self.bias_).flatten()
            
    def predict(self, X):
        return np.sign(self.predict_proba(X))

    def draw(self, ax_i, ax_j, *, title=None, axis_names=None):
        '''
        Plots cool beautiful shockingly magnificent graph.
        I spent 5 hours figuring this out, so yeah.

        Params:
        ax_i and ax_j is coordinates for picture in plt.subplots
        Title is title, and axis_names is the names of the axises
        
        '''

        xvals = np.linspace(self.cords_[:, 0].min() - 1, self.cords_[:, 0].max() + 1, int(self.M_))
        yvals = np.linspace(self.cords_[:, 1].min() - 1, self.cords_[:, 1].max() + 1, int(self.M_))

        space = np.meshgrid(xvals, yvals)
        #print(space)
        space = np.column_stack((space[0].ravel(), space[1].ravel()))
        if self.transform == 'square':
            space = square_transform(space)
        elif self.transform == 'cube':
            space = cube_transform(space)

        # zz = []
        # for m in range(self.M_ ** 2):
        #     zz.append(svm.predict_proba(space[m]))

        zz = self.predict_proba(space)
        xx, yy = xvals, yvals
        
        zz = np.array(zz).reshape(int(self.M_), int(self.M_))
        ax[ax_i, ax_j].pcolormesh(xx, yy, zz, shading='gouraud', cmap='viridis')

        margin = 1 / np.linalg.norm(self.weights_)

        ax[ax_i, ax_j].contour(xx, yy, zz, levels=(0), colors='k', linewidths=1.5, zorder=1)
        ax[ax_i, ax_j].contour(xx, yy, zz, levels=([-1, 1]), colors='white', linewidths=1.5, zorder=2, linestyles='dotted')
        
        ax[ax_i, ax_j].scatter(self.cords_[:, 0], self.cords_[:, 1], marker="o", c=self.labels_, s=25, cmap='viridis', edgecolor="k");
        
        support_vectors = []
        support_labels = []
        for i in range(self.M_):
            if dot_line_distanse(self.weights_, self.bias_, self.cords_[i]) <= margin + 10 ** -4:
                support_vectors.append(self.cords_[i])
                support_labels.append(self.labels_[i])
        support_vectors = np.array(support_vectors)

        if len(support_vectors):
            ax[ax_i, ax_j].scatter(support_vectors[:, 0], support_vectors[:, 1], marker="o", c=support_labels,
                                   s=25, cmap='viridis', edgecolor="1")
        
        ax[ax_i, ax_j].set_ylim(self.cords_[:, 1].min() - 1, self.cords_[:, 1].max() + 1)
        ax[ax_i, ax_j].set_title(title)

# 3. Двойственный переход и Ядровой SVM

Задачу обучения линейного SVM, рассмотренную в предыдущем пункте принято называть _прямой_ оптимизационной задачей для SVM. Любая задача оптимизации с ограничениями имеет [_двойственную_ задачу Лагранжа](http://goo.gl/OujTPr), в которой оптимизируются _двойственные переменные_ (множители Лагранжа), соответствующие штрафу за нарушение ограничений, максимизируется нижняя оценка функционала прямой задачи. В случае задачи квадратичного программирования, решение двойственной задачи (значение оптимизируемого функционала) совпадает с оптимумом прямой задачи.

Подробнее можно почитать в [статье](./SMAIS11_SVM.pdf).

Двойственная задача для SVM имеет вид:
$$
\begin{gather}
    \sum_{n} \alpha_n - \frac{1}{2}\sum_{n}\sum_{n'} \alpha_{n}\alpha_{n'} y_{n}y_{n'} x_{n}^Tx_{n'} \to \max_{\alpha} \\
    \begin{aligned}
        \text{s.t. } \quad  
        & 0 \le \alpha_n \le C, \quad \forall n = 1, \dots, N \\
        & \sum_{n} \alpha_n y_n = 0
    \end{aligned}
\end{gather}
$$

Оптимизируется вектор из двойственных переменных $\alpha_n$, соответствующих объектам обучающей выборки. Объект $x_n$ является опорным, если $\alpha_n > 0$.

Предсказание вычисляется по следующему правилу:
$$\hat{y}(x) = \text{sign}\left(\sum_{n}\alpha_{n}y_{n}x^Tx_{n} + b\right).$$

Для предсказания необходимо оценить значение $b$. Известно, что для любого опорного объекта, который классифицируется безошибочно верно:
$$y_n = \sum_{n'}\alpha_{n}y_{n}x_{n}^Tx_{n'} + b,$$
значит для любого такого объекта:
$$b = y_n - \sum_{n'}\alpha_{n}y_{n}x_{n}^Tx_{n'}.$$

В случае наличия ошибок классификации обучающей выборки, предлагается усреднять значение $b$ по всем опорным векторам:
$$b = \frac{1}{N_\text{SV}}\sum_{n \in \text{SV}}\left(y_n - \sum_{n'}\alpha_{n}y_{n}x_{n}^Tx_{n'}\right).$$
Интуиция здесь такова, что суммарные ошибки в положительную сторону примерно равны суммарным ошибкам в отрицательную сторону.

Другой вариант — отказаться от параметра $b$ и работать с моделью $f(x) = w^Tx$, добавив к вектору $x$ константный признак.

#### Неявное преобразование признаков
Отметим, что двойственная задача SVM содержит вектора признаков исключительно в виде скалярного произведения $x^Tx'$. Эта особенность позволяет производить неявное преобразование признакового пространства. Вместо вычисления функции $\phi(\boldsymbol x)$, которая может отображать исходные признаки в вектора очень большой размерности, будем вычислять скалярное произведение $k(\boldsymbol x, \boldsymbol x') = \phi(\boldsymbol x)^T\phi(\boldsymbol x')$ называемое _ядром_. 

#### Задание
  1. Реализуйте процедуру обучения ядрового SVM, используя QP-солвер.
  2. Протестируйте на случайных двумерных выборках ядровой SVM. Покажите на плоскости строящиеся разделяющие поверхности, линии уровня, ограничивающие коридор $f(\boldsymbol x) = \pm 1$. Выделите опорные вектора точками другой формы или большего размера. Попробуйте следующие ядровые функции:
    - линейная: $k(x, x') = x^Tx'$
    - полиномиальная: $k(x, x') = (x^Tx' + 1)^p$ с различными степенями $p = 2,3,\dots$
    - Гауссовская-RBF: $k(x, x') = \exp(-\frac{1}{2\gamma}\|x - x'\|^2)$
  3. Как ведет себя SVM с полиномиальным ядром в зависимости от параметров $C$ и степени ядра $p$?
  4. Как ведет себя SVM с RBF-ядром в зависимости от параметров $C$ и $\gamma$? Поварьируйте параметры $C$ и $\gamma$ по логарифмической сетке. Какие значения параметров ведут к переобучению, а какие — к слишком грубой модели?

# Примеры

Настройка вывода графиков [`Maplotlib`](http://matplotlib.org/) и импорт функций из модуля [`pylab`](http://wiki.scipy.org/PyLab).

In [22]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['matrix']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Визуальное решение задачи квадратичного программирования

На следующем рисунке наглядно показано решение задачи QP из задания 1. Оптимизируемая функция $f(\boldsymbol x)$ показана линиями уровня, область значений недопустимых ограничениями окрашена в серый цвет.

In [26]:
np.array([True, False, False]) & np.array([True, False, True])

array([ True, False, False])

In [27]:
np.logical_and(np.array([True, False, False]), np.array([True, False, True]))

array([ True, False, False])

## Установка и использование `CvxOpt`

Библиотека [`cvxopt`](http://cvxopt.org/) может быть установлена как обычный python-пакет:

In [5]:
!pip install --upgrade --user cvxopt

  Using cached https://files.pythonhosted.org/packages/16/a0/0d090735e2639a74d6628831e02cc59284e3a3a4f5910f496fc6e435b645/cvxopt-1.2.5-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/56/39/537cb3e4e93f1ac5085dc3b3a43cfd99d0af9b29c44fcaa99490f526b611/mkl-2019.0-py2.py3-none-win_amd64.whl


Нас будет интересовать функция [`cvxopt.solvers.qp()`](http://cvxopt.org/examples/tutorial/qp.html):